In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

warnings.filterwarnings("ignore")

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# Objetivo del cuadernillo

En este cuadernillo se quiere realizar el preprocesamiento de los datos de egresos hospitalarios para confeccionar un modelo de previsión de esta métrica hasta el 2035. De forma general, se utilizarán los datos desde 2001 hasta 2018 para entrenar modelos, y 2019 para evaluar los modelos. Se obviará 2020 del análisis debido a la naturaleza del año (pandemia).

- Se entrenarán 6 modelos distintos, y se seleccionará el que tenga el mejor desempeño en el conjunto de evaluación (año 2019). Los modelos a entrenar serán:

    - Prophet
    - Auto ARIMA
    - XGB
    - Hibrido
    - Feed Forward Neural Network
    - Long-Short Term Memory (LSTM)

Para simplificar el modelamiento de los datos, se harán las predicciones con una agregación de egresos mensuales.

In [3]:
# Lee la base de datos
egresos_torax = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")

# Crea columna de fecha en datetime
egresos_torax["FECHA_EGRESO"] = pd.to_datetime(
    egresos_torax["ANO_EGRESO"].astype(str)
    + "-"
    + egresos_torax["MES_EGRESO"].astype(str)
    + "-"
    + egresos_torax["DIA_EGRESO"].astype(str),
    format="%Y-%m-%d",
)

# Elimina columnas redundantes
egresos_torax = egresos_torax.drop(
    columns=[
        "ANO_EGRESO",
        "MES_EGRESO",
        "DIA_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "GLOSA_ESTABLECIMIENTO_SALUD",
    ]
).sort_values("FECHA_EGRESO")

# Elimina el anio 2020, debido a que es anomalo (Pandemia COVID)
egresos_torax = egresos_torax.query("FECHA_EGRESO < '2020-01-01'")

# Elige la fecha como indice de los datos
egresos_torax = egresos_torax.set_index("FECHA_EGRESO")

In [5]:
DIAGS_MAS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I080",
    "I081",
    "I340",
    "I350",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J398",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J90X",
    "J91X",
    "J931",
    "J955",
    "J980",
    "M348",
    "Q211",
    "Q231",
    "Q676",
    "T820",
    "T821",
    "Z450",
]

In [8]:
# Obtiene metricas agregadas por mes (suma los egresos de todos los diags por dia)
df = egresos_torax.resample("M").sum()[["n_egresos"]]
df_relevantes = (
    egresos_torax[egresos_torax["DIAG1"].isin(DIAGS_MAS_RELEVANTES)]
    .resample("M")
    .sum()[["n_egresos"]]
)

# 1. Preprocesamiento de los datos

Cada uno de los modelos requiere un preprocesamiento distinto de los datos. Por lo tanto, se creará un apartado para cada uno.

In [35]:
# Previo a cualquier preprocesamiento, se separa en train y test
CORTE_TEST = "2019-01-01"

train = df[df.index < CORTE_TEST]
test = df[df.index >= CORTE_TEST]

## Prophet

In [36]:
train_prophet = train.reset_index().rename(columns={"FECHA_EGRESO": "ds", "n_egresos": "y"})
test_prophet = test.reset_index().rename(columns={"FECHA_EGRESO": "ds", "n_egresos": "y"})

## Auto ARIMA

In [37]:
train_auto_arima = train["n_egresos"]
test_auto_arima = test["n_egresos"]

## XGB

In [114]:
train_xgb = build_features.add_time_series_columns(train)
test_xgb = build_features.add_time_series_columns(test)